# PDF Text Extraction

## 1. Packages

In [1]:
import os
import json
from pypdf import PdfReader
from openai import OpenAI #import openai
from IPython.display import display, JSON
import tiktoken

- Principios **SOLID**, la estructura de una **función en Python**

    ```python
    def procesar_pedido(pedido: Pedido, validador: ValidadorPedido, notificador: Notificador) -> bool:
        """
        Procesa un pedido asegurando que cumple con las validaciones necesarias antes de ser aprobado.

        WARNING:

        Args:
            pedido (Pedido): Objeto que representa el pedido.
            validador (ValidadorPedido): Objeto que maneja la validación del pedido.
            notificador (Notificador): Servicio para enviar notificaciones.

        Returns:
            bool: True si el pedido fue procesado con éxito, False en caso contrario.
        """
        if not validador.validar(pedido):
            return False  # Retorno temprano en caso de error

        pedido.marcar_como_procesado()
        notificador.enviar_confirmacion(pedido)
        
        return True  # Indica éxito
    ```

## 2. OpenAI Client Instantiation

In [2]:
# Recuperar la clave API de la variable de entorno
api_key_environ = os.environ.get("OPENAI_API_KEY")

# Verificar que la clave API esté disponible
if not api_key_environ:
    raise ValueError("La variable de entorno OPENAI_API_KEY no está configurada o está vacía.")

# Inicializar el cliente de OpenAI con la clave API
client = OpenAI(api_key=api_key_environ)

# Usar el cliente para tus tareas
print("¡Cliente de OpenAI inicializado correctamente!")

ValueError: La variable de entorno OPENAI_API_KEY no está configurada o está vacía.

## 3. Variables used

In [83]:
# Ruta local de la carpeta con PDFs
carpeta_local = "../assets/DG_docs/"

## 4. Functions

### ~~def extraer_texto_pdf(ruta_pdf)-> str:~~
### DESCARTADA: No se tiene en cuenta la referencia a que página pertenece el texto

In [39]:
# Función depende del paquete pypdf

# def extraer_texto_pdf(ruta_pdf)-> str:
#     """
#     Extrae el texto de un archivo PDF usando pypdf.

#     WARNING:
#         No se tiene en cuenta la referencia a que página pertenece el texto

#     Args:
#         ruta_pdf: Ruta en donde están los PDFs a proecesar

#     Returns:
#         bool: True si el pedido fue procesado con éxito, False en caso contrario.

#     Example:
#         temp_texto = extraer_texto_pdf('../assets/DG_docs/2024S-VBOG-030188.pdf')
#         print(temp_texto)
#     """
#     texto = ""
#     try:
#         reader = PdfReader(ruta_pdf)
#         for page in reader.pages:
#             texto += page.extract_text() or ""  # Extrae texto por página
#     except Exception as e:
#         print(f"Error leyendo {ruta_pdf}: {e}")
#     return texto

In [40]:
# # Ejemplo de uso
# temp_texto = extraer_texto_pdf('../assets/DG_docs/2025S-VBOG-001950.pdf') #2025S-VBOG-001950.pdf 2024S-VBOG-030188.pdf
# display(temp_texto)
# display("El número de caracteres del texto extraido es:", len(temp_texto))

### def count_tokens(text: str, encodig_name: str = "text-embedding-3-small") -> int:

In [45]:
# Función depende de `tiktoken`
def count_tokens(text: str, encodig_name: str = "text-embedding-3-small") -> int:
    """
    Cuenta la cantidad de tokens en un texto usando el modelo de OpenAI.

    Packages:
        import tiktoken

    Args:
        texto:  Texto a procesar.
        modelo: Modelo de OpenAI para la tokenización (por defecto "text-embedding-3-small").
    
    Returns:
        int: Cantidad de tokens en el texto.
    """
    try:
        encoding = tiktoken.encoding_for_model(encodig_name)
        tokens = encoding.encode(text)
        print(f"\n\t{len(text)} caracteres del archivo \n\t{len(tokens)} tokens basado en el modelo {encodig_name}")
        return len(tokens)
    except Exception as e:
        print(f"Error contando tokens: {e}")
        return -1  # Devuelve -1 en caso de error


# # contabilizar la cantidad de tokens que se enviarían a OpenAI

# import tiktoken

# def num_tokens_from_string(text, encodig_name):
#     encoding = tiktoken.get_encoding(encodig_name)
#     num_tokens = len(encoding.encode(text))
#     return num_tokens


In [48]:
# # Ejemplo de uso

# texto = "Este es un ejemplo de texto para contar tokens."
# #texto = extraer_texto_pdf(ruta_pdf)
# num_tokens = count_tokens(texto)


	47 caracteres del archivo 
	10 tokens basado en el modelo text-embedding-3-small


### def generar_embedding(texto: str) -> list:

[Cómo los modelos para embeddings como `sentence-transforme` y `ChatGpt` manejan inputs muy grandes](../AI_Queries/prompt_AI_GPT-Transformer-how_works.md)

OpenAI limita a 8192 tokens por entrada

#### ~~Pendiente: Asegurar que en los `embeddings` se procese todo;~~

In [81]:

def generar_embedding(texto: str) -> list:
    """
    Genera un embedding del texto usando OpenAI y lo estructura en un objeto JSON.

    WARNING:
        
    Packages:
        from openai import OpenAI
        import os
        import json

    Args:
        texto: Texto a procesar para obtener su embedding.

    Returns:
        dict: Un diccionario con el embedding generado y la cantidad de tokens procesados.

    Example:
        generar_embedding("Este es un texto de prueba")
    """
    
    try:
        # Llama a la API de OpenAI para obtener embeddings
        client = OpenAI()
        respuesta = client.embeddings.create(
            input=texto[:8191],  # OpenAI limita a 8192 tokens por entrada
            # EL COMENTARIO ANTERIOR ESTA MAL ENFORACADO PORQUE SE RECORTA EL TEXTO A 8191 Y LO QUE SE DEBE RECORTAR ES HASTA 8191 TOKENS
            model="text-embedding-3-small"  # Modelo optimizado para embeddings
        )
    
    except Exception as e:
        print(f"Error generando embedding: {e}")
        return None
    
    return respuesta.data[0].embedding


### extraer_texto_pdf(ruta_pdf) -> dict:

CAMBIAR EL NOMBRE DE LA FUNCIÓN POR ALGO REALCIONADO CON EXTRAER TEXTO, ESTRUCTURARLO DEN FORMATO JSON Y GUARDARLO EN UN ARCHIVO .JSON

In [75]:
# Función depende de
#   PACKAGE:  json, os, pypdf(PdfReader)
#   FUNCTION: count_tokens, generar_embedding

def extraer_texto_pdf(ruta_pdf: str) -> dict:
    """
    Extrae el texto de un archivo PDF y lo estructura en un objeto JSON.

    WARNING:
        

    Requirements:
        PACKAGE:  json, os, pypdf(PdfReader)
        FUNCTION: count_tokens, generar_embedding

    Args:
        ruta_pdf: Ruta del archivo PDF a procesar

    Returns:
        dict: Un diccionario con el contenido extraído de cada página del PDF y lo guarda en un archivo JSON.

    Example:
        extraer_texto_pdf('../assets/DG_docs/2024S-VBOG-030188.pdf')
    """
    nombre_archivo = os.path.basename(ruta_pdf)
    resultado = {"name": nombre_archivo, "text": []}
    
    try:
        reader = PdfReader(ruta_pdf)
        
        for num_pagina, page in enumerate(reader.pages, start=1):
            texto_pagina = page.extract_text() or ""
            resultado["text"].append({
                "page": num_pagina,
                "content": texto_pagina.strip(),
                "tokens": count_tokens(texto_pagina),
                "OpenAI_embedding" : (lambda texto: None if texto == "" else generar_embedding(texto_pagina))(texto_pagina)
                #  generar_embedding(texto_pagina)
            })

        
        # Validar que el JSON sea correcto antes de guardarlo
        json_str = json.dumps(resultado, indent=4, ensure_ascii=False)

        nombre_json = os.path.splitext(nombre_archivo)[0] + ".json"
        
        # Guardar el resultado en un archivo JSON
        with open(nombre_json, "w", encoding="utf-8") as json_file:
            json_file.write(json_str)
    
    except Exception as e:
        print(f"Error leyendo {ruta_pdf} o guardando el JSON: {e}")
        return None
    
    return resultado


In [ ]:
# # Ejemplo de uso
# temp_texto = extraer_texto_pdf('../assets/DG_docs/2025S-VBOG-001950.pdf') #2025S-VBOG-001950.pdf 2024S-VBOG-030188.pdf
# display(JSON(temp_texto))
# # FALTA CREAR UNA FUNCIÓN QUE CUENTE LOS CARACTERES DEL PDF, RECORRIENDO EL "content" DE CADA "page" DE CADA "file"
# #display("El número de caracteres del texto extraido es:", len(temp_texto))


	2839 caracteres del archivo 
	1010 tokens basado en el modelo text-embedding-3-small

	3217 caracteres del archivo 
	924 tokens basado en el modelo text-embedding-3-small

	3206 caracteres del archivo 
	1023 tokens basado en el modelo text-embedding-3-small

	29 caracteres del archivo 
	11 tokens basado en el modelo text-embedding-3-small


<IPython.core.display.JSON object>

### ~~guardar_texto_en_archivo(texto: str, ruta_salida: str)-> bool:~~
### DESCARTADA: Porque la función `def extraer_texto_pdf(ruta_pdf: str) -> dict:` ya guarda el texto y las caracteristicas del archivo en un .json

In [6]:
# # Funcion sin dependencias

# def guardar_texto_en_archivo(texto: str, ruta_salida: str)-> bool:
#     """
#     Guarda el texto extraído en un archivo de texto plano.

#     Args:
#         texto:          Texto a guardar.
#         ruta_salida:    Ruta del archivo de salida.
    
#     Returns:
#         bool: True si se desarrollló el proceso de guardado del texto correctamente, False en caso  que `with open` genere error.
#     """
#     try:
#         with open(ruta_salida, "w", encoding="utf-8") as archivo:
#             archivo.write(texto)
#         print(f"Texto guardado en {ruta_salida}")
#         return True
#     except Exception as e:
#         print(f"Error guardando el archivo ({ruta_salida}): {e}")
#         return False

In [ ]:
# # Ejemplo de uso
# ruta_pdf = "../assets/DG_docs/2025S-VBOG-001950.pdf"
# ruta_txt = "../assets/DG_docs/2025S-VBOG-001950.txt"
# texto_extraido = extraer_texto_pdf(ruta_pdf)
# #display(len(texto_extraido))
# guardar_texto_en_archivo(texto_extraido, ruta_txt)

### PRUEBA

In [8]:
encoding = tiktoken.encoding_for_model("text-embedding-3-small")
'''
- text-embedding-3-large $0.065
-> text-embedding-3-small ----------> este es el 'Embeddings' más económico $0.01
- text-embedding-ada-002 $0.05
https://platform.openai.com/docs/models#embeddings
https://platform.openai.com/docs/pricing
'''
tokens = encoding.encode(texto_extraido)
tokens_recortados = tokens[:8192]  # Limitamos a 8192 tokens
#display(tokens_recortados)
texto_recortado = encoding.decode(tokens_recortados)  # Convertimos de vuelta a texto
#display(texto_recortado)

In [88]:
# Función depende de `os`

def folder_processing(carpeta: str) -> list:
    """
    Procesa todos los archivos PDF en una carpeta y genera sus respectivos embeddings.

    Packages:
        import os

    Args:
        carpeta (str): Ruta de la carpeta que contiene los archivos PDF.

    Returns:
        list: Lista de diccionarios con el nombre del archivo y su embedding.
    
    Example:
        embeddings = folder_processing("ruta/de/la/carpeta")
    """
    resultados = []
    try:
        archivos_pdf = [archivo for archivo in os.listdir(carpeta) if archivo.endswith(".pdf")]
        print(f"\nSe encontraron {len(archivos_pdf)} archivos PDF en la carpeta '{carpeta}'.")

        for archivo in archivos_pdf:
            ruta_pdf = os.path.join(carpeta, archivo)
            print(f"\n🔹 Procesando: {archivo}")

            texto = extraer_texto_pdf(ruta_pdf)
            if not texto:
                print(f"⚠️ No se pudo extraer texto de {archivo}. Saltando...")
                continue

            # embedding = generar_embedding(texto)
            # if embedding:
            #     resultados.append({"archivo": archivo, "embedding": embedding})
            #     print(f"✅ Embedding generado para {archivo} ({len(embedding)} dimensiones).")
            # else:
            #     print(f"⚠️ Falló la generación de embedding para {archivo}.")
        
        # print(f"\n🔍 Procesamiento finalizado: {len(resultados)} archivos procesados correctamente.")

    except Exception as e:
        print(f"\n Error procesando la carpeta '{carpeta}': {e}")

    return resultados


In [89]:
folder_processing(carpeta_local)


Se encontraron 4 archivos PDF en la carpeta '../assets/DG_docs/'.

🔹 Procesando: 2024S-VBOG-030188.pdf

	2066 caracteres del archivo 
	661 tokens basado en el modelo text-embedding-3-small

	1962 caracteres del archivo 
	643 tokens basado en el modelo text-embedding-3-small

	2010 caracteres del archivo 
	646 tokens basado en el modelo text-embedding-3-small

	2185 caracteres del archivo 
	669 tokens basado en el modelo text-embedding-3-small

	2659 caracteres del archivo 
	884 tokens basado en el modelo text-embedding-3-small

	2215 caracteres del archivo 
	795 tokens basado en el modelo text-embedding-3-small

	2026 caracteres del archivo 
	812 tokens basado en el modelo text-embedding-3-small

	2068 caracteres del archivo 
	842 tokens basado en el modelo text-embedding-3-small

	2062 caracteres del archivo 
	795 tokens basado en el modelo text-embedding-3-small

	2247 caracteres del archivo 
	913 tokens basado en el modelo text-embedding-3-small

	2795 caracteres del archivo 
	943 

[]

2024S-VBOG-030188.pdf

TRANVERSAL DEL CUSIANA

RUTA LIBERTADORES

PERDIDA DE UN 80% DE LA VÍA

Contrato de obra

In [35]:
consulta = (["RUTA DE LOS COMUNEROS", "CORREDOR LA SOBERANIA"])
resultados_consulta = []
# resultados_consulta.append({"archivo": archivo, "embedding": embedding})

# with open("embeddings_query.json", "w", encoding="utf-8") as f:
#     json.dump(resultados_consulta, f, ensure_ascii=False, indent=4)

## Otros

In [ ]:
# ejemplo práctico de clear_output(wait=True) en Jupyter Notebook
# Simula datos que se actualizan sin mostrar varias tablas a la vez.

import pandas as pd
from IPython.display import display, clear_output
import time

data = {"Producto": ["Laptop", "Teléfono", "Tablet"], "Stock": [10, 15, 8]}

for i in range(5):
    clear_output(wait=True)  # Borra la tabla anterior
    data["Stock"] = [x - 1 for x in data["Stock"]]  # Simula reducción de stock
    df = pd.DataFrame(data)
    display(df)  # Muestra la tabla actualizada
    time.sleep(1)

,Producto,Stock
0,Laptop,5
1,Teléfono,10
2,Tablet,3


In [ ]:
os.listdir("../assets/DG_docs/")

['2024S-VBOG-030188.pdf',
 '2024S-VBOG-030188.txt',
 '2024S-VBOG-054632_X.pdf',
 '2024S-VBOG-057351_X.pdf',
 '2025S-VBOG-001950.pdf',
 '2025S-VBOG-001950.txt',
 'Nueva carpeta']